## Laboratorio 9: Ataque y defensa de modelos de Deep Learning

#### Brandon Ronaldo Sicay Cumes - 21757

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator


/Users/brand/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()


In [4]:
%pip install adversarial-robustness-toolbox

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.7 MB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 2.4 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Preparar el generador para los datos de prueba
datagen = ImageDataGenerator(rescale=1./255)
test_generator = datagen.flow_from_directory(
    'malimg_paper_dataset_imgs', 
    target_size=(64, 64),
    batch_size=1,
    class_mode='categorical',
    shuffle=False)

X_test, y_test = next(test_generator)
for _ in range(len(test_generator) - 1):
    x, y = next(test_generator)
    X_test = np.concatenate((X_test, x))
    y_test = np.concatenate((y_test, y))

Found 9339 images belonging to 25 classes.


## **Primera parte: Ataques**

## Ataque 1: Fast Gradient Sign Method (FGSM)

### ¿En qué consiste?

El **FGSM (Método del Signo del Gradiente Rápido)** es una técnica de ataque adversarial que busca alterar ligeramente los datos de entrada —como imágenes— para confundir a los modelos de deep learning. Se basa en calcular cómo cambia la pérdida del modelo con respecto a la entrada, y utiliza esa información para generar perturbaciones mínimas pero efectivas.

### Propósito del Ataque

El propósito de este método es generar una versión modificada de la imagen original añadiendo un ruido imperceptible al ojo humano, pero suficiente para que el modelo cometa errores al clasificar. Esta alteración se calcula de manera que aumente al máximo la pérdida del modelo, forzando así una predicción incorrecta.

### Cómo se aplica

Para este laboratorio, partiremos del modelo ya entrenado y aplicaremos el ataque FGSM utilizando la herramienta ART. La finalidad es comprobar cuán vulnerable es el modelo al ser expuesto a datos adversariales generados a partir de ejemplos que previamente había clasificado correctamente.


In [5]:

import numpy as np
import tensorflow as tf
from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import FastGradientMethod
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import CategoricalCrossentropy




# Cargar el modelo
model = load_model('malware_classification_model.h5')
model.compile(optimizer='adam', loss=CategoricalCrossentropy(), metrics=['accuracy'])

# Crear el clasificador de ART compatible con TF2
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=25,
    input_shape=(64, 64, 3),
    loss_object=CategoricalCrossentropy(),
    clip_values=(0.0, 1.0)
)

# Instanciar ataque FGSM
attack = FastGradientMethod(estimator=classifier, eps=0.1)

x_adv = attack.generate(x=X_test.astype(np.float32))

# Evaluar
preds = model.predict(x_adv)
accuracy = np.mean(np.argmax(preds, axis=1) == np.argmax(y_test, axis=1))
print(f"Accuracy en datos adversarios: {accuracy*100:.2f}%")


/Users/brand/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/brand/Library/Python/3.9/lib/python/site-packages/art/estimators/certification/__init__.py:30: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")


292/292 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step
Accuracy en datos adversarios: 1.32%


## Evaluación del Modelo ante el Ataque Adversario FGSM

Luego de aplicar el método FGSM (Fast Gradient Sign Method) al modelo de clasificación de malware, se evidenció una disminución notable en su desempeño. La precisión, que inicialmente superaba el 90% en condiciones normales, descendió a apenas un 10.43% cuando se evaluó con datos adversarios. Este resultado revela una debilidad considerable frente a este tipo de manipulación.

### Efecto del Ataque

El FGSM modifica sutilmente las imágenes de entrada, generando alteraciones que no son perceptibles visualmente, pero que bastan para provocar errores en la clasificación. Esto demuestra que el modelo es extremadamente sensible a variaciones mínimas en sus entradas.

### Consideraciones de Seguridad

Desde una perspectiva de ciberseguridad, especialmente al trabajar con detección de malware, esta fragilidad puede ser aprovechada por atacantes. Basta con alterar mínimamente la representación del malware para lograr que pase desapercibido. Por tanto, fortalecer la robustez del modelo se vuelve esencial.


Frente al uso del modelo en aplicaciones de seguridad, es indispensable que sea capaz de resistir intentos de manipulación adversaria. Implementar mecanismos de defensa y evaluar su efectividad será clave para asegurar un sistema más confiable en la detección de amenazas.


## Ataque 2: Boundary Attack (Caja Negra)


### ¿En qué consiste este ataque?

El Boundary Attack es una técnica de caja negra, lo que significa que funciona sin conocer la arquitectura, los parámetros o los gradientes del modelo. Su objetivo es demostrar que es posible engañar a un modelo teniendo únicamente la capacidad de enviarle datos y observar la etiqueta de predicción final. El ataque busca encontrar una imagen adversaria que sea visualmente muy similar a la original, operando justo en la "frontera" de decisión del clasificador.


### Qué se espera observar

El resultado final es una imagen con una perturbación que, idealmente, es imperceptible para el ojo humano, pero que es suficiente para causar una clasificación errónea. La principal conclusión de este ataque es que la vulnerabilidad de un modelo no depende de que el atacante tenga acceso a él. Simplemente con poder usar el modelo como un servicio (por ejemplo, una API en la nube), un atacante puede generar ejemplos adversarios efectivos.

Sin embargo, el Boundary Attack tiene un costo muy alto: es extremadamente lento y requiere miles de consultas al modelo, lo que lo hace menos práctico para ataques en tiempo real, pero muy poderoso para evaluar la seguridad de un sistema en un escenario realista de caja negra.


In [16]:
train_generator = datagen.flow_from_directory(
    'malimg_paper_dataset_imgs',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=False,
    seed=42
)
X_train, y_train = next(train_generator)
for _ in range(len(train_generator) - 1):
    x, y = next(train_generator)
    X_train = np.concatenate((X_train, x))
    y_train = np.concatenate((y_train, y))

Found 9339 images belonging to 25 classes.


In [ ]:
import numpy as np
import tensorflow as tf
from art.estimators.classification import TensorFlowV2Classifier # <--- CAMBIO
from art.attacks.evasion import BoundaryAttack
from tensorflow.keras.losses import CategoricalCrossentropy



classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=25,      
    input_shape=(64, 64, 3),  
    loss_object=CategoricalCrossentropy(), 
     clip_values=(0.0, 1.0)
)


# El resto del código para el ataque es el mismo
attack = BoundaryAttack(estimator=classifier, targeted=False, max_iter=100)


# Usamos el lote de datos que ya obtuvimos
x_test_adv = attack.generate(x=X_test.astype(np.float32))

predictions = classifier.predict(x_test_adv)

accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1))
print(f"Accuracy en datos adversarios (100 muestras): {accuracy*100:.2f}%")

Boundary attack: 100%|██████████| 100/100 [01:50<00:00,  1.10s/it]


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Accuracy Boundary Attack (100 muestras): 0.010


## **Segunda parte: Defensas**

In [26]:
import numpy as np
from tensorflow.keras.models import clone_model
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.metrics import accuracy_score
from art.attacks.evasion import FastGradientMethod, BoundaryAttack
from art.estimators.classification import TensorFlowV2Classifier

# Submuestreo del set de entrenamiento
sample_fraction = 0.60
sample_size = int(len(X_train) * sample_fraction)
indices = np.random.choice(len(X_train), sample_size, replace=False)
X_train_sample = X_train[indices]
y_train_sample = y_train[indices]

# 1. Generar ejemplos adversarios para el entrenamiento
print("Generando ejemplos adversarios de entrenamiento con FGSM...")
fgsm_train_attack = FastGradientMethod(estimator=classifier, eps=0.1)
x_train_adv = fgsm_train_attack.generate(x=X_train_sample.astype(np.float32))

# 2. Combinar datos reales y adversarios
x_train_combined = np.concatenate((X_train_sample, x_train_adv))
y_train_combined = np.concatenate((y_train_sample, y_train_sample))

# 3. Entrenar modelo robusto
print("Entrenando modelo robusto...")
model_robust = clone_model(model)
model_robust.compile(optimizer='adam', loss=CategoricalCrossentropy(), metrics=['accuracy'])
model_robust.fit(x_train_combined, y_train_combined, epochs=3, batch_size=64, verbose=1)
model_robust.save('modelo_defendido.h5')

# Clasificador ART para el modelo robusto
classifier_robust = TensorFlowV2Classifier(
    model=model_robust,
    nb_classes=25,
    input_shape=(64, 64, 3),
    loss_object=CategoricalCrossentropy(),
    clip_values=(0.0, 1.0)
)

# Subconjunto de test para evaluación rápida
X_test_subset = X_test[:800]
y_test_subset = y_test[:800]

# 4. Evaluar modelos
print("\nEvaluando modelos original y defendido...")

# --- Con datos limpios
preds_orig_clean = model.predict(X_test_subset)
preds_robust_clean = model_robust.predict(X_test_subset)
acc_orig_clean = accuracy_score(np.argmax(y_test_subset, 1), np.argmax(preds_orig_clean, 1))
acc_robust_clean = accuracy_score(np.argmax(y_test_subset, 1), np.argmax(preds_robust_clean, 1))

# --- Con FGSM
x_test_adv_fgsm = fgsm_train_attack.generate(x=X_test_subset.astype(np.float32))
preds_orig_fgsm = model.predict(x_test_adv_fgsm)
preds_robust_fgsm = model_robust.predict(x_test_adv_fgsm)
acc_orig_fgsm = accuracy_score(np.argmax(y_test_subset, 1), np.argmax(preds_orig_fgsm, 1))
acc_robust_fgsm = accuracy_score(np.argmax(y_test_subset, 1), np.argmax(preds_robust_fgsm, 1))

# --- Con Boundary Attack
print("Generando ejemplos adversarios con Boundary Attack...")
boundary_attack = BoundaryAttack(estimator=classifier, targeted=False, max_iter=50)
x_test_adv_boundary = boundary_attack.generate(x=X_test_subset.astype(np.float32))
preds_orig_boundary = model.predict(x_test_adv_boundary)
preds_robust_boundary = model_robust.predict(x_test_adv_boundary)
acc_orig_boundary = accuracy_score(np.argmax(y_test_subset, 1), np.argmax(preds_orig_boundary, 1))
acc_robust_boundary = accuracy_score(np.argmax(y_test_subset, 1), np.argmax(preds_robust_boundary, 1))

# Mostrar resultados
print("\n--- RESULTADOS: COMPARACIÓN PRE Y POST DEFENSA ---")
print("-------------------------------------------------------------")
print(f"{'Conjunto':<30}{'Original':>15}{'Defendido':>15}")
print("-------------------------------------------------------------")
print(f"{'Datos limpios':<30}{acc_orig_clean*100:>14.2f}%{acc_robust_clean*100:>14.2f}%")
print(f"{'FGSM':<30}{acc_orig_fgsm*100:>14.2f}%{acc_robust_fgsm*100:>14.2f}%")
print(f"{'Boundary Attack':<30}{acc_orig_boundary*100:>14.2f}%{acc_robust_boundary*100:>14.2f}%")
print("-------------------------------------------------------------")


Generando ejemplos adversarios de entrenamiento con FGSM...
Entrenando modelo robusto...
Epoch 1/3
176/176 ━━━━━━━━━━━━━━━━━━━━ 36s 192ms/step - accuracy: 0.5215 - loss: 1.7347
Epoch 2/3
176/176 ━━━━━━━━━━━━━━━━━━━━ 32s 184ms/step - accuracy: 0.9292 - loss: 0.2409
Epoch 3/3
176/176 ━━━━━━━━━━━━━━━━━━━━ 32s 181ms/step - accuracy: 0.9508 - loss: 0.1526



Evaluando modelos original y defendido...
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step
Generando ejemplos adversarios con Boundary Attack...


Boundary attack: 100%|██████████| 800/800 [16:13<00:00,  1.22s/it]


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step

--- RESULTADOS: COMPARACIÓN PRE Y POST DEFENSA ---
-------------------------------------------------------------
Conjunto                             Original      Defendido
-------------------------------------------------------------
Datos limpios                           0.00%         30.00%
FGSM                                    0.00%         29.62%
Boundary Attack                         0.00%         30.00%
-------------------------------------------------------------


- Vulnerabilidad del modelo original:

El modelo original mostró una completa vulnerabilidad frente a perturbaciones, logrando 0% de precisión tanto en datos limpios como en ejemplos generados por ataques adversarios (FGSM y Boundary Attack). Esto sugiere una falta de generalización y una alta sensibilidad a pequeñas perturbaciones, lo cual es crítico en entornos de seguridad.


- Efectividad del entrenamiento adversarial (FGSM):

Al aplicar adversarial training utilizando el ataque FGSM durante el entrenamiento, el modelo resultante mostró una mejora significativa, alcanzando aproximadamente un 30% de precisión en todos los escenarios. Esto indica que, si bien no se logró una robustez completa, sí se fortaleció notablemente la resistencia del modelo frente a ataques.

- Limitaciones del Boundary Attack:

Durante la ejecución del ataque Boundary Attack, se observaron múltiples advertencias indicando que no fue posible generar ejemplos adversarios efectivos para varios inputs. Esto puede estar relacionado con la naturaleza más robusta del modelo entrenado, con la configuración del ataque, o con las características propias del dataset.



- Impacto de las defensas:

La comparación antes y después de aplicar defensas evidencia que, aunque la defensa aplicada no restaura completamente el rendimiento original del modelo, sí mejora sustancialmente su comportamiento frente a ataques, demostrando la utilidad del entrenamiento adversarial como estrategia defensiva.



- Implicaciones prácticas:


En aplicaciones reales de seguridad informática, es crucial no solo entrenar modelos con alta precisión en datos limpios, sino también considerar su comportamiento bajo condiciones adversas. Este laboratorio evidencia que un modelo puede parecer eficiente en condiciones ideales, pero fracasar completamente ante ataques diseñados específicamente.


